In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
from sklearn.model_selection import cross_val_score

ImportError: No module named model_selection

In [ ]:
df_train = pd.read_csv('../data/train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('../data/test.csv', encoding="ISO-8859-1")
# df_attr = pd.read_csv('../input/attributes.csv')
df_pro_desc = pd.read_csv('../data/product_descriptions.csv')
num_train = df_train.shape[0]

In [ ]:
def str_stemmer(s):
	return " ".join([stemmer.stem(word) for word in s.lower().split()])

def str_common_word(str1, str2):
	words , count = str1.split(),0
	for word in words:
	    if str2.find(word)>0:
	        count+=1
	return count

def categorize_target(x):
    if x is None:
        return None
    if x <= 2:
        return 0
    return 1

In [ ]:
df_pro_desc.head()

In [4]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')

df_all['search_term'] = df_all['search_term'].map(lambda x:str_stemmer(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stemmer(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stemmer(x))
df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title']+"\t"+df_all['product_description']
df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all = df_all.drop(['search_term','product_title','product_description','product_info'],axis=1)

In [5]:
df_all['relevance'] = df_all['relevance'].map(lambda x:categorize_target(x))
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']

y_train = df_train['relevance'].values
X_train = df_train.drop(['id','relevance'],axis=1).values
X_test = df_test.drop(['id','relevance'],axis=1).values

In [8]:
print('Building model ....')
rf = RandomForestClassifier(n_estimators=100, max_depth=3,random_state=0)
#clf = BaggingClassifier(rf, n_estimators=60, max_samples=0.2, random_state=25)
lr = LogisticRegression()
nb = GaussianNB()

scores = cross_val_score(lr, X_train, y_train, cv=10)
print scores
print scores.mean()

Building model ....
[ 0.68084245  0.68084245  0.68084245  0.68084245  0.68084245  0.68084245
  0.68084245  0.68084245  0.68079935  0.68089129]
0.680843021222
